In [87]:
import jsonlines
import datetime
import json
import requests
import numpy as np
import os
import pandas as pd
from requests_oauthlib import OAuth1

In [91]:
api_creds = {
    'consumer_key':'',
    'consumer_secret':'',
    'access_token':'',
    'access_token_secret':''
}

auth = OAuth1(api_creds['consumer_key'], api_creds['consumer_secret'],
                     api_creds['access_token'],api_creds['access_token_secret'])

In [92]:
def getUserTweets(screen_name,authorization,fetchAttempts):
    
    api_url='https://api.twitter.com/1.1/statuses/user_timeline.json?screen_name={}&count=200&trim_user=true&tweet_mode=extended'
    api_url=api_url.format(screen_name)
    tweets = []
    for i in range(fetchAttempts):
        tweets = tweets + requests.get(api_url,auth=authorization).json()
    return tweets

def saveJsonline(tweetList,filename):
    with open(filename,'w',encoding="utf-8") as file:
        book = jsonlines.Writer(file)
        book.write_all(tweetList)
        book.close

In [93]:
tweets = getUserTweets('',auth,3)
print(len(tweets))
saveJsonline(tweets,'tweets.jsonl')

600


In [94]:
def tweetInfo(tweet):
    cleanTweet = []
    for col in ['created_at','full_text', 'favorite_count','retweet_count']:
        cleanTweet.append(tweet[col])
    
    cleanTweet[0] = datetime.datetime.strptime(cleanTweet[0][:-10]+cleanTweet[0][-4:],"%a %b %d %H:%M:%S %Y")
    count=0
    if 'extended_entities' in tweet.keys():
        if 'media' in tweet['extended_entities'].keys():
            for m in tweet['extended_entities']['media']:
                if m['type'] == 'photo':
                    count = count + 1
    
    if(count == 0):
        cleanTweet.append(None)
    else:
        cleanTweet.append(count)
    
    return cleanTweet

In [97]:
def TweetsTable(tweets):
    table = [tweetInfo(t) for t in tweets]  
    df = pd.DataFrame(table,columns=['Timestamp','Tweets','Likes','Retweets','Image Count'])
    pd.options.display.max_rows
    pd.set_option('display.max_colwidth', -1)
    return df

#display(TweetsTable(tweets))